In [16]:
import pandas as pd
df = pd.read_csv('asmr-20240924.csv', encoding='Windows-1252', sep=';', on_bad_lines='skip')
print("\nFirst 5 rows of the DataFrame:")
print(df.head())


First 5 rows of the DataFrame:
  $Code Evamed$        $Motif de demande$  $Code CIS$       $Code CIP$  \
0    $CT-21025$        $Inscription (CT)$  $66077218$  $3400930294611$   
1    $CT-21016$        $Inscription (CT)$  $64314227$  $3400955101734$   
2    $CT-21007$        $Inscription (CT)$  $60120260$  $3400930280522$   
3    $CT-20978$        $Inscription (CT)$  $63696560$  $3400930283073$   
4    $CT-20969$  $Extension d'indication$  $63363591$  $3400930247556$   

                           $Dénomination spécialité$ $Date avis définitif$  \
0    $FEXOFENADINE ARROW 180 mg, comprimé pelliculé$          $04/09/2024$   
1  $OXYGENE MEDICINAL LINDE HEALTHCARE 200 bar, g...          $28/08/2024$   
2  $CHLORHEXIDINE LIBERTY PHARMA 0,20 %, solution...          $28/08/2024$   
3  $EBGLYSS 250 mg, solution injectable en sering...          $04/09/2024$   
4  $CHLORAPREP, tampon cutané imprégné en applica...          $04/09/2024$   

   $ASMR$ $Valeur ASMR$                               

In [17]:
import pandas as pd
df = pd.read_csv('asmr-20240924.csv', encoding='Windows-1252', sep=';', on_bad_lines='skip')
print(f"Total number of columns: {df.shape[1]}")
print(f"Column names: {df.columns}")


Total number of columns: 9
Column names: Index(['$Code Evamed$', '$Motif de demande$', '$Code CIS$', '$Code CIP$',
       '$Dénomination spécialité$', '$Date avis définitif$', '$ASMR$',
       '$Valeur ASMR$', '$Libellé ASMR$ '],
      dtype='object')


In [20]:
import pandas as pd
df = pd.read_csv('asmr-20240924.csv', encoding='Windows-1252', sep=';', on_bad_lines='skip')
if df.shape[1] >= 9:  # Ensure there are at least 9 columns
    columns_8 = df.iloc[:100, [7]]
    print(columns_8)
else:
    print("The DataFrame has fewer than 9 columns.")

   $Valeur ASMR$
0            $V$
1            $V$
2            $V$
3            $V$
4            $V$
..           ...
95           $V$
96          $IV$
97          $IV$
98           $V$
99           $V$

[100 rows x 1 columns]


In [2]:
import pandas as pd
path = ('2023_descriptif-variables_open-medic.xls')
xl = pd.ExcelFile(path)
print(xl.sheet_names)


['Variables', 'ATC1', 'ATC2', 'ATC3', 'ATC4', 'ATC5', 'CIP13', 'TOP_GEN', 'GEN_NUM', 'AGE', 'SEXE', 'BEN_REG', 'PSP_SPE']


In [5]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
if not os.environ.get("LANGCHAIN_API_KEY"):
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

 ········


In [22]:
import pandas as pd

df = pd.read_csv('asmr-20240924.csv', encoding='Windows-1252', sep=';', on_bad_lines='skip')
print(df.shape)
print(df.columns.tolist())

(9243, 9)
['$Code Evamed$', '$Motif de demande$', '$Code CIS$', '$Code CIP$', '$Dénomination spécialité$', '$Date avis définitif$', '$ASMR$', '$Valeur ASMR$', '$Libellé ASMR$ ']


In [27]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

engine = create_engine("sqlite:///asmr.db")
df.to_sql("asmr", engine, index=False)

9243

In [29]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
print(db.run("SELECT * FROM asmr WHERE Age < 2;"))

sqlite
['asmr', 'titanic']


OperationalError: (sqlite3.OperationalError) no such column: Age
[SQL: SELECT * FROM asmr WHERE Age < 2;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [8]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

 ········


In [9]:
from langchain_community.agent_toolkits import create_sql_agent

agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [10]:
agent_executor.invoke({"input": "what's the average age of survivors"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


titanic
Invoking: `sql_db_schema` with `{'table_names': 'titanic'}`



CREATE TABLE titanic (
	"Survived" BIGINT, 
	"Pclass" BIGINT, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" FLOAT, 
	"Siblings/Spouses Aboard" BIGINT, 
	"Parents/Children Aboard" BIGINT, 
	"Fare" FLOAT
)

/*
3 rows from titanic table:
Survived	Pclass	Name	Sex	Age	Siblings/Spouses Aboard	Parents/Children Aboard	Fare
0	3	Mr. Owen Harris Braund	male	22.0	1	0	7.25
1	1	Mrs. John Bradley (Florence Briggs Thayer) Cumings	female	38.0	1	0	71.2833
1	3	Miss. Laina Heikkinen	female	26.0	0	0	7.925
*/
Invoking: `sql_db_query` with `{'query': 'SELECT AVG(Age) AS Average_Age FROM titanic WHERE Survived = 1'}`


[(28.408391812865496,)]The average age of survivors is approximately 28.41 years.

> Finished chain.


{'input': "what's the average age of survivors",
 'output': 'The average age of survivors is approximately 28.41 years.'}